Packages Installation
======
Try to run the first cell to see where it crash but normally if you start from a empty virtual python environment this is what is required :

    python -m pip install -r requirements.txt

with these lines inside the "requirements.txt" file:

    ipykernel==6.29.5
    pandas==2.2.2
    matplotlib==3.9.2
    plotly==5.22.0
    dash==2.17.1

Importing python packages 

In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math

from dash import Dash, html, dash_table, dcc, callback, Output, Input


import plotly.graph_objs as go

Interractive Plot of Fortescue Transform 
======

In [8]:
# Initialize the Dash app
app = Dash(__name__)
frequency_b = 50. #Hz
voltage_b = 320 #V (230*2**0.5)
power_b = 1e6 #W

# Function to generate three-phase signals
def generate_three_phase_signals(amplitudes,phase_shifts):
    

    nb_of_periods = 2.5 #number of periods to simulate
    duration = nb_of_periods * (1/frequency_b) #seconds

    sampling_frequency = 1e5 #Hz
    sampling_period = 1/sampling_frequency #seconds

    t = np.arange(0,duration,sampling_period) #time vector seconds
    omega = 2 * np.pi * frequency_b

    # Generate three-phase signals with phase shifts
    va = math.sqrt(2)*amplitudes[0]/voltage_b*np.sin(omega * t + np.radians(phase_shifts[0]))
    vb = math.sqrt(2)*amplitudes[1]/voltage_b*np.sin(omega * t + np.radians(phase_shifts[1]))
    vc = math.sqrt(2)*amplitudes[2]/voltage_b*np.sin(omega * t + np.radians(phase_shifts[2]))

    return t, va, vb, vc

def compute_abc_phasors(amplitudes,phase_shifts):
    # Compute the phasors for each phase
    va_phasor = amplitudes[0]/voltage_b  * np.exp(1j * np.radians(phase_shifts[0]))
    vb_phasor = amplitudes[1]/voltage_b  * np.exp(1j * np.radians(phase_shifts[1]))
    vc_phasor = amplitudes[2]/voltage_b  * np.exp(1j * np.radians(phase_shifts[2]))
    return va_phasor, vb_phasor, vc_phasor

def get_amplitudes_and_phases(phasors):
    # Extract amplitudes and phases from the phasors
    amplitudes = np.abs(phasors)
    phases = np.angle(phasors, deg=True)
    return amplitudes, phases

def get_sum_phasors(phasors):
    # Compute the sum of the phasors
    sum_phasors = np.sum(phasors, axis=0)
    return sum_phasors

# Function to compute Positive-Negative-Zero sequence components
def compute_pnz_sequence(v_phasor = np.array([0,0,0])):
    a_coeff =-1/2 +1j*(3**0.5 /2) #a coefficient for the Fortescue transformation
    a_coeff_squared = np.conjugate(a_coeff) #a coefficient squared for the Fortescue transformation 
    Fortescue_mat = np.ones((3,3),dtype=complex)
    Fortescue_mat[0,1] = a_coeff
    Fortescue_mat[0,2] = a_coeff_squared
    Fortescue_mat[1,1] = a_coeff_squared
    Fortescue_mat[1,2] = a_coeff
    print(f"Fortescue :{Fortescue_mat}")
    Fortescue_mat_norm = 1/3*Fortescue_mat #normalization factor for the Fortescue transformation
    print(f"Fortescue Final norm :{Fortescue_mat_norm}")
    sequence_phasor = Fortescue_mat_norm @ v_phasor
    return sequence_phasor

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Interactive Electrical Sequence Components Visualization"),
    
    # Input fields for Amplitudes
    html.Div([
        html.Label("Amplitude for Phase A (V):"),
        dcc.Input(id='amplitude-a', type='number', value=320),
        html.Label("Amplitude for Phase B (V):"),
        dcc.Input(id='amplitude-b', type='number', value=320),
        html.Label("Amplitude for Phase C (V):"),
        dcc.Input(id='amplitude-c', type='number', value=320),
    ], style={'margin-bottom': '20px'}),
    

    # Input fields for phase shifts
    html.Div([
        html.Label("Phase Shift for Phase A (degrees):"),
        dcc.Input(id='phase-a', type='number', value=0),
        html.Label("Phase Shift for Phase B (degrees):"),
        dcc.Input(id='phase-b', type='number', value=-120),
        html.Label("Phase Shift for Phase C (degrees):"),
        dcc.Input(id='phase-c', type='number', value=120),
    ], style={'margin-bottom': '20px'}),

    # Graphs
    dcc.Graph(id='three-phase-plot'),
    html.Div([
        dcc.Graph(id='phasor-abc-plot', style={'display': 'inline-block', 'width': '50%'}),
        dcc.Graph(id='phasor-pnz-plot', style={'display': 'inline-block', 'width': '50%'}),
        
    ]),
    #dcc.Graph(id='phasor-abc-plot'),
    # Separate sequence component phasor diagrams
    html.Div([
        dcc.Graph(id='phasor-positive-plot', style={'display': 'inline-block', 'width': '33%'}),
        dcc.Graph(id='phasor-negative-plot', style={'display': 'inline-block', 'width': '33%'}),
        dcc.Graph(id='phasor-zero-plot', style={'display': 'inline-block', 'width': '33%'}),
    ]),
])

# Callback to update the plots
@app.callback(
    [Output('three-phase-plot', 'figure'),
     Output('phasor-abc-plot', 'figure'),
     Output('phasor-pnz-plot', 'figure'),
     Output('phasor-positive-plot', 'figure'),
     Output('phasor-negative-plot', 'figure'),
     Output('phasor-zero-plot', 'figure')],
    [Input('phase-a', 'value'),
     Input('phase-b', 'value'),
     Input('phase-c', 'value'),
     Input('amplitude-a', 'value'),
     Input('amplitude-b', 'value'),
     Input('amplitude-c', 'value')]
)
def update_plots(phase_a, phase_b, phase_c, amplitude_a, amplitude_b, amplitude_c):
    phases = np.array([phase_a, phase_b, phase_c])
    amplitudes = np.array([amplitude_a, amplitude_b, amplitude_c])
    # Generate signals
    t, va, vb, vc = generate_three_phase_signals(amplitudes,phases)
    va_phasor, vb_phasor, vc_phasor = compute_abc_phasors(amplitudes, phases)

    phasors_abc = np.array([va_phasor, vb_phasor, vc_phasor])

    # Compute PNZ sequence components
    phasors_pnz = compute_pnz_sequence(phasors_abc)
    amplitudes_pnz, phases_pnz = get_amplitudes_and_phases(phasors_pnz)
    # Compute the sum of the phasors
    sum_phasors = get_sum_phasors(phasors_pnz)
    magnitude_sum = np.abs(sum_phasors)
    # Time-domain plot
    time_fig = go.Figure()
    time_fig.add_trace(go.Scatter(x=t, y=va, mode='lines', name='Phase A'))
    time_fig.add_trace(go.Scatter(x=t, y=vb, mode='lines', name='Phase B'))
    time_fig.add_trace(go.Scatter(x=t, y=vc, mode='lines', name='Phase C'))
    time_fig.update_layout(title="Three-Phase Voltage Signals", xaxis_title="Time (s)", yaxis_title="Amplitude (pu)")
    
    # Phasor diagram in ABC frame
    abc_fig = go.Figure()
    abc_fig.add_trace(go.Scatterpolar(r=[amplitude_a/voltage_b , 0], theta=[phase_a, 0], mode='lines+markers', name='Phase A'))
    abc_fig.add_trace(go.Scatterpolar(r=[amplitude_b/voltage_b , 0], theta=[phase_b, 0], mode='lines+markers', name='Phase B'))
    abc_fig.add_trace(go.Scatterpolar(r=[amplitude_c/voltage_b , 0], theta=[phase_c, 0], mode='lines+markers', name='Phase C'))
    abc_fig.update_layout(title="Phasor Diagram (ABC Frame) - (pu)(deg°)", polar=dict(angularaxis=dict(rotation=0),radialaxis=dict(range=[0, 1.5])))
    
    # Phasor diagram in PNZ frame
    pnz_fig = go.Figure()
    pnz_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[0], 0], theta=[phases_pnz[0], 0], mode='lines+markers', name='Positive Sequence'))
    pnz_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[1], 0], theta=[phases_pnz[1], 0], mode='lines+markers', name='Negative Sequence'))
    pnz_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[2], 0], theta=[phases_pnz[2], 0], mode='lines+markers', name='Zero Sequence'))
    #pnz_fig.add_trace(go.Scatterpolar(r=[magnitude_sum, 0], theta=[np.angle(sum_phasors, deg=True), 0], mode='lines+markers', name='Sum Phasor'))
    pnz_fig.update_layout(title="Phasor Diagram (PNZ Frame) - (pu)(deg°)", 
                            polar=dict(
                            angularaxis=dict(rotation=0),
                            radialaxis=dict(range=[0, 1.5])  # Set the radial axis range (adjust as needed)
    ))
    # #Display the magnitude of the sum phasor on the plot
    # pnz_fig.add_annotation(
    #     text=f"Sum Phasor Magnitude: {magnitude_sum:.2f}",
    #     x=0, y=0, showarrow=True, arrowhead=2,
    #     ax=0, ay=-40, font=dict(size=12), bgcolor="white"
    # )
    
    # Zero sequence phasor diagram
    zero_fig = go.Figure()
    zero_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[2], 0], theta=[phases_pnz[2], 0], mode='lines+markers', name='Zero Sequence'))
    zero_fig.update_layout(title="Zero Sequence Phasor", polar=dict(angularaxis=dict(rotation=0), radialaxis=dict(range=[0, 1.5])))

    # Positive sequence phasor diagram
    positive_fig = go.Figure()
    positive_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[0], 0], theta=[phases_pnz[0], 0], mode='lines+markers', name='Positive Sequence A'))
    positive_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[0], 0], theta=[phases_pnz[0]-120, 0], mode='lines+markers', name='Positive Sequence B'))
    positive_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[0], 0], theta=[phases_pnz[0]+120, 0], mode='lines+markers', name='Positive Sequence C'))
    positive_fig.update_layout(title="Positive Sequence Phasor", polar=dict(angularaxis=dict(rotation=0), radialaxis=dict(range=[0, 1.5])))
    
    # Negative sequence phasor diagram
    negative_fig = go.Figure()
    negative_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[1], 0], theta=[phases_pnz[1], 0], mode='lines+markers', name='Negative Sequence A'))
    negative_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[1], 0], theta=[phases_pnz[1]+120, 0], mode='lines+markers', name='Negative Sequence B'))
    negative_fig.add_trace(go.Scatterpolar(r=[amplitudes_pnz[1], 0], theta=[phases_pnz[1]-120, 0], mode='lines+markers', name='Negative Sequence C'))
    negative_fig.update_layout(title="Negative Sequence Phasor", polar=dict(angularaxis=dict(rotation=0), radialaxis=dict(range=[0, 1.5])))
    
    
    
    return time_fig, abc_fig, pnz_fig, zero_fig, positive_fig, negative_fig,

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8051)

Fortescue :[[ 1. +0.j        -0.5+0.8660254j -0.5-0.8660254j]
 [ 1. +0.j        -0.5-0.8660254j -0.5+0.8660254j]
 [ 1. +0.j         1. +0.j         1. +0.j       ]]
Fortescue Final norm :[[ 0.33333333+0.j         -0.16666667+0.28867513j -0.16666667-0.28867513j]
 [ 0.33333333+0.j         -0.16666667-0.28867513j -0.16666667+0.28867513j]
 [ 0.33333333+0.j          0.33333333+0.j          0.33333333+0.j        ]]
